# O/D Efficiency

Efficiency is calculated by +1 for being on the line when we score and -1 for being on the line when we get scored on, then normalized.

Use 'Points vs. ...' csv for what happened on each point and 'Player Stats vs. ...' for who was on the line for that point

In [2]:
%pylab inline
import pandas as pd
from os import listdir
from os.path import isfile, join

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Offensive/Deffensive Officiency

# single player single game

def get_game_data(game):
	# returns a dataframe for the points csv and the points played for each player
	def points_played_to_array(row):
		data = row['Points played'].split(',')
		return pd.Series({'Player': row['Player'], 'Points played': np.array([int(point) for point in data])})\

	player_stats_df = pd.read_csv(mypath + 'Player Stats vs. ' + game)
	points_played_df = player_stats_df.apply(points_played_to_array, axis=1)

	return pd.read_csv(mypath + 'Points vs. ' + game), points_played_df

def o_eff(player, game):
	# return raw player offensive score, total o points played
	points_df, points_played_df = get_game_data(game)
	
	plus = 0
	minus = 0

	if player in points_played_df['Player'].unique():
		plus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
								  points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 1)]['Point']))

		minus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
								   points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 0)]['Point']))
	return plus - minus, plus + minus

def total_o_eff(player, games):
	# return the total offensive efficiency for a player over a list of games
	o_plus_minus = 0
	o_points_played = 0
	for game in games:
		new_plus_minus, new_points_played = o_eff(player, game)
		o_plus_minus += new_plus_minus
		o_points_played += new_points_played

	if o_points_played == 0:
		return None
	else:
		return o_plus_minus/o_points_played

def d_eff(player, game):
	# return raw player defensive score, total d points played
	points_df, points_played_df = get_game_data(game)

	plus = 0
	minus = 0

	if player in points_played_df['Player'].unique():
		plus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
								  points_df[(points_df['Started on offense?'] == 0) & (points_df['Scored?'] == 1)]['Point']))

		minus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
								   points_df[(points_df['Started on offense?'] == 0) & (points_df['Scored?'] == 0)]['Point']))
	return plus - minus, plus + minus

def total_d_eff(player, games):
	# return the total offensive efficiency for a player over a list of games
	d_plus_minus = 0
	d_points_played = 0
	for game in games:
		new_plus_minus, new_points_played = d_eff(player, game)
		d_plus_minus += new_plus_minus
		d_points_played += new_points_played
	if d_points_played == 0:
		return None
	else:
		return d_plus_minus/d_points_played

mypath = 'WY-Wolves/'

players_df = pd.read_csv(mypath + '1617 SUIS Wolves Players.csv')
players = list(players_df['Player'])

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
games = []
for f in onlyfiles:
	if('vs.' in f):
		idx = f.index('vs.')
		games.append(f[idx + 4:])

games = list(set(games))
rows = []
for player in players:
	row = {'Player': player, 'O eff': total_o_eff(player, games), 'D eff': total_d_eff(player, games)}
	rows.append(row)

eff = pd.DataFrame(rows)
print(eff.sort_values('O eff', ascending=False))
print(eff.sort_values('D eff', ascending=False))

       D eff     O eff            Player
0   0.200000  1.000000       #7 Cici Qiu
9   0.636364  1.000000    #39 David Xiao
1   1.000000  1.000000        #77 Zoe Wu
2   0.310345  0.555556     #2 Jamie Chen
29  0.578947  0.500000   #11 Momoka Goto
16  0.250000  0.428571      #76 Jack Xia
10  0.555556  0.428571    #30 Jimmy Peng
27  0.378049  0.393258  #27 Daniel Zhang
24  0.214286  0.391304      #41 Billy Ni
19  0.154930  0.370370   #69 Justin Chen
20  0.333333  0.369565    #14 Cecilia Wu
28  0.357143  0.333333      #0 Julie Luo
22  0.666667  0.333333       Khloris Liu
33  0.500000  0.333333             Peter
4   0.430769  0.318182  #22 Paul Simpson
8   0.333333  0.314286      #28 Tina Zhu
26  0.279503  0.295238     #1 Joyce Wang
7   0.465116  0.290323      #19 Ben Wang
17  0.444444  0.276596     #55 Tigger Li
30  0.100000  0.263158     Theresa Zhang
23  0.031250  0.250000    #66 Jerry Chen
15  0.312500  0.133333      Melody Zheng
5   1.000000  0.000000           Tony Ma
3   0.571429  0.

In [4]:
mypath = 'WY-Wolves/'
players_df = pd.read_csv(mypath + '1617 SUIS Wolves Players.csv')
players = list(players_df['Player'].unique())
players

['#7 Cici Qiu',
 '#77 Zoe Wu',
 '#2 Jamie Chen',
 '#36 Jason Ling',
 '#22 Paul Simpson',
 'Tony Ma',
 'Susan Zhang',
 '#19 Ben Wang',
 '#28 Tina Zhu',
 '#39 David Xiao',
 '#30 Jimmy Peng',
 'Milaney Mi',
 '#25 Alex Chen',
 'Peach',
 'Daniel',
 'Melody Zheng',
 '#76 Jack Xia',
 '#55 Tigger Li',
 '#29 Tony Wu',
 '#69 Justin Chen',
 '#14 Cecilia Wu',
 '#52 Emily Zhang',
 'Khloris Liu',
 '#66 Jerry Chen',
 '#41 Billy Ni',
 'Oscar Yao',
 '#1 Joyce Wang',
 '#27 Daniel Zhang',
 '#0 Julie Luo',
 '#11 Momoka Goto',
 'Theresa Zhang',
 '#86 Ellis Tan',
 'Chris Xu',
 'Peter',
 'Bruce Gu']

In [14]:
game = 'DQ-Cheetahs 2017-02-24_16-15.csv'
player_stats_df = pd.read_csv(mypath + 'Player Stats vs. ' + game)
player_stats_df


,Player,Points played total,Points played,Touches,Points played with touches,Throws,Catches,Possessions initiated,Assists,Secondary assists,...,Receiver errors,Defensive blocks,Stall outs for,Stall outs against,Average completed throw distance (m),Average completed throw gain (m),Average incomplete throw distance (m),Average incomplete throw gain (m),Average caught pass distance (m),Average caught pass gain (m)
0,#55 Tigger Li,13,"2,3,4,6,7,8,9,10,12,13,14,15,16",43,13,42,24,19,7,0,...,0,3,0,0,21.71,14.67,29.81,26.96,14.48,6.52
1,#14 Cecilia Wu,13,"1,3,4,5,7,8,9,10,11,12,13,16,17",18,9,18,14,4,0,0,...,1,4,0,0,14.23,6.10,18.69,15.25,12.34,2.97
2,#1 Joyce Wang,11,"2,4,5,7,9,10,11,14,15,16,17",9,4,9,8,1,0,1,...,1,1,0,0,13.37,8.75,0.00,0.00,15.15,4.32
3,#22 Paul Simpson,11,"1,3,4,6,7,9,10,11,14,15,17",30,8,27,15,15,1,3,...,1,5,0,0,19.10,11.76,34.72,33.93,15.48,5.65
4,Melody Zheng,11,"1,3,4,5,7,10,11,14,15,16,17",1,1,1,1,0,0,0,...,1,0,0,0,0.00,0.00,11.33,-8.54,18.83,17.95
5,Theresa Zhang,8,"1,3,6,8,12,13,16,17",1,1,1,1,0,0,1,...,0,0,0,0,10.21,-0.74,0.00,0.00,17.52,17.33
6,#0 Julie Luo,8,"2,6,8,9,14,15,16,17",0,0,0,0,0,0,0,...,1,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
7,#69 Justin Chen,7,"1,5,7,10,11,14,15",6,5,3,5,1,0,0,...,2,2,0,0,12.74,12.50,15.29,13.61,31.38,27.05
8,Susan Zhang,6,"2,6,12,13,16,17",1,1,1,1,0,0,0,...,1,0,0,0,12.44,12.38,0.00,0.00,34.82,27.60
9,#27 Daniel Zhang,6,"5,8,10,11,12,13",10,5,7,7,3,1,0,...,1,1,0,0,14.27,8.94,27.23,20.05,28.22,25.11


In [102]:
player_stats_df['Player'].unique()
player = '#1 Joyce Wang'
player in player_stats_df['Player'].unique()
def points_to_set_of_ints(row):
    points = row['Points played'].split(',')
    #player = row['Player']
    #data = np.array([int(point) for point in points])
    #print(data)
    #print(np.intersect1d(data, np.array([1, 2, 3])))
    return pd.Series({'Player': row['Player'], 'Points played': np.array([int(point) for point in points])})

points_played_df = player_stats_df.apply(points_to_set_of_ints, axis=1)
np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0], [1, 2, 3])

    
#set(player_stats_df.iloc[player_stats_df[player_stats_df['Player'] == player].index]['Points played'].tolist()[0].split(','))

array([2])

In [94]:
points_played = points_played_df[points_played_df['Player'] == player]['Points played'].values[0]
points_played

array([ 2,  4,  5,  7,  9, 10, 11, 14, 15, 16, 17])

In [28]:
points_df = pd.read_csv(mypath + 'Points vs. ' + game)
points_df

,Created,Point,Our score at pull,Opponent's score at pull,Started on offense?,Scored?,Possessions,Passes,Turnovers,Thrower errors,Receiver errors,Defensive blocks,Opposition errors,Secondary assist,Assist,Goal
0,2017-02-24 16:05,1,0,0,1,0,3,8,3,2,2,1,1,NaN,NaN,NaN
1,2017-02-24 16:17,2,0,1,1,0,6,8,6,4,4,1,4,NaN,NaN,NaN
2,2017-02-24 16:24,3,0,2,1,0,2,5,2,1,1,0,1,NaN,NaN,NaN
3,2017-02-24 16:27,4,0,3,1,1,1,5,0,0,0,0,0,#22 Paul Simpson,Peter,#22 Paul Simpson
4,2017-02-24 16:30,5,1,3,0,0,2,3,2,1,1,0,2,NaN,NaN,NaN
5,2017-02-24 16:34,6,1,4,1,1,1,7,0,0,0,0,0,Theresa Zhang,#22 Paul Simpson,#55 Tigger Li
6,2017-02-24 16:37,7,2,4,0,1,4,10,3,3,0,4,0,NaN,#55 Tigger Li,#22 Paul Simpson
7,2017-02-24 16:43,8,3,4,0,1,1,7,0,0,0,0,1,Peter,#55 Tigger Li,#27 Daniel Zhang
8,2017-02-24 16:46,9,4,4,0,0,3,11,3,2,1,1,2,NaN,NaN,NaN
9,2017-02-24 16:51,10,4,5,1,1,2,10,1,1,0,0,1,#1 Joyce Wang,#55 Tigger Li,#69 Justin Chen


In [105]:
np.intersect1d(points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 1)]['Point'],points_played)

array([ 4, 10], dtype=int64)

In [107]:
plus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
                          points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 1)]['Point']))
plus

2

In [108]:
minus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
                          points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 0)]['Point']))
minus

2

In [110]:
def o_eff(player, game):
	# return raw player offensive score, total o points played
	player_stats_df = pd.read_csv(mypath + 'Player Stats vs. ' + game)
	points_df = pd.read_csv(mypath + 'Points vs. ' + game)

	def points_played_to_set(row):
		data = row['Points played'].split(',')
		return pd.Series({'Player': row['Player'], 'Points played': np.array([int(point) for point in data])})

	points_played_df = player_stats_df.apply(points_played_to_set, axis=1)

	plus = 0
	minus = 0

	if player in player_stats_df['Player'].unique():
		plus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
								  points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 1)]['Point']))

		minus = len(np.intersect1d(points_played_df[points_played_df['Player'] == player]['Points played'].values[0],
								   points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 0)]['Point']))
	return plus - minus, plus + minus

o_eff(player, game)


(0, 4)

In [3]:
player_stats_df = pd.read_csv('WY-Wolves/Player Stats vs. YK-Buddha 2017-03-11_12-30.csv')
points_df = pd.read_csv('WY-Wolves/Points vs. YK-Buddha 2017-03-11_12-30.csv')

In [4]:
player_stats_df['Player']

0     #22 Paul Simpson
1        #1 Joyce Wang
2       #14 Cecilia Wu
3     #27 Daniel Zhang
4        #2 Jamie Chen
5      #69 Justin Chen
6       #66 Jerry Chen
7         #28 Tina Zhu
8         Melody Zheng
9        #25 Alex Chen
10           Oscar Yao
11        #19 Ben Wang
12       Theresa Zhang
13        #41 Billy Ni
14        #0 Julie Luo
15        #76 Jack Xia
Name: Player, dtype: object

In [4]:
player_stats_df = player_stats_df.set_index('Player')

In [5]:
player_stats_df

,Points played total,Points played,Touches,Points played with touches,Throws,Catches,Possessions initiated,Assists,Secondary assists,Goals,...,Receiver errors,Defensive blocks,Stall outs for,Stall outs against,Average completed throw distance (m),Average completed throw gain (m),Average incomplete throw distance (m),Average incomplete throw gain (m),Average caught pass distance (m),Average caught pass gain (m)
Player,,,,,,,,,,,,,,,,,,,,,
Joyce Wang,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",16,10,14,15,1,3,2,2,...,0,1,0,0,16.27,13.51,16.75,16.21,16.06,10.52
#22 Paul Simpson,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",40,15,39,18,22,5,2,1,...,1,0,0,0,17.14,11.37,27.74,24.17,15.09,7.26
#27 Daniel Zhang,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",12,8,11,11,1,1,4,1,...,1,2,0,0,19.68,13.55,42.87,40.59,12.68,8.17
Cecilia Wu,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",11,7,11,9,2,0,0,0,...,0,1,0,0,14.79,5.98,26.43,26.36,12.85,3.19
#69 Justin Chen,12,"1,2,3,7,8,9,10,12,13,14,15,16",6,6,4,5,1,0,0,2,...,0,2,0,0,11.35,0.17,32.26,22.40,20.94,11.53
Jamie Chen,10,"1,2,3,4,5,6,9,10,13,14",5,5,3,5,0,1,0,2,...,2,1,0,0,19.30,16.83,19.97,18.01,21.83,17.60
Tina Zhu,10,"3,4,5,6,7,8,11,12,15,16",6,5,6,6,0,0,0,0,...,0,0,0,0,9.16,-2.70,16.21,10.27,14.44,6.48
#19 Ben Wang,4,"2,11,15,16",2,2,1,2,0,0,0,1,...,1,0,0,0,7.58,-2.72,0.00,0.00,23.72,23.64
Theresa Zhang,4,"9,10,13,14",0,0,0,0,0,0,0,0,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00


In [16]:
player = 'Joyce Wang'
points_played = player_stats_df.loc[player]['Points played'].split(',')
points_played = [int(item) for item in points_played]
points_played = pd.Series(points_played)
points_played

0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10
10    11
11    12
12    13
13    14
14    15
15    16
dtype: int64

In [17]:
o_points_scored = points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 1)]
o_points_scored

,Created,Point,Our score at pull,Opponent's score at pull,Started on offense?,Scored?,Possessions,Passes,Turnovers,Thrower errors,Receiver errors,Defensive blocks,Opposition errors,Secondary assist,Assist,Goal
2,2017-03-11 14:51,3,0,2,1,1,1,9,0,0,0,0,0,Joyce Wang,#27 Daniel Zhang,Jamie Chen
6,2017-03-11 15:01,7,3,3,1,1,2,7,1,1,0,0,1,#41 Billy Ni,#22 Paul Simpson,#69 Justin Chen
8,2017-03-11 15:07,9,4,4,1,1,1,7,0,0,0,0,0,#27 Daniel Zhang,Joyce Wang,Jamie Chen
12,2017-03-11 15:21,13,7,5,1,1,1,6,0,0,0,0,0,#22 Paul Simpson,Jamie Chen,#69 Justin Chen
14,2017-03-11 15:25,15,8,6,1,1,2,8,1,1,0,1,0,Joyce Wang,#22 Paul Simpson,Joyce Wang


In [18]:
o_points_scored['Point']

2      3
6      7
8      9
12    13
14    15
Name: Point, dtype: int64

In [19]:
# I want to know the length of the series of o_points_scored where player played that point (point is in points_played)
plus = len(list(set(o_points_scored['Point']) & set(points_played)))

In [20]:
o_points_lost = points_df[(points_df['Started on offense?'] == 1) & (points_df['Scored?'] == 0)]
o_points_lost

,Created,Point,Our score at pull,Opponent's score at pull,Started on offense?,Scored?,Possessions,Passes,Turnovers,Thrower errors,Receiver errors,Defensive blocks,Opposition errors,Secondary assist,Assist,Goal
1,2017-03-11 14:46,2,0,1,1,0,4,13,4,3,1,0,3,NaN,NaN,NaN


In [21]:
minus = len(list(set(o_points_lost['Point']) & set(points_played)))

In [22]:
o_eff = (plus - minus)/(plus + minus)
o_eff

0.6666666666666666

In [26]:
player_stats_df.reset_index()

,Player,Points played total,Points played,Touches,Points played with touches,Throws,Catches,Possessions initiated,Assists,Secondary assists,...,Receiver errors,Defensive blocks,Stall outs for,Stall outs against,Average completed throw distance (m),Average completed throw gain (m),Average incomplete throw distance (m),Average incomplete throw gain (m),Average caught pass distance (m),Average caught pass gain (m)
0,Joyce Wang,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",16,10,14,15,1,3,2,...,0,1,0,0,16.27,13.51,16.75,16.21,16.06,10.52
1,#22 Paul Simpson,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",40,15,39,18,22,5,2,...,1,0,0,0,17.14,11.37,27.74,24.17,15.09,7.26
2,#27 Daniel Zhang,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",12,8,11,11,1,1,4,...,1,2,0,0,19.68,13.55,42.87,40.59,12.68,8.17
3,Cecilia Wu,16,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16",11,7,11,9,2,0,0,...,0,1,0,0,14.79,5.98,26.43,26.36,12.85,3.19
4,#69 Justin Chen,12,"1,2,3,7,8,9,10,12,13,14,15,16",6,6,4,5,1,0,0,...,0,2,0,0,11.35,0.17,32.26,22.40,20.94,11.53
5,Jamie Chen,10,"1,2,3,4,5,6,9,10,13,14",5,5,3,5,0,1,0,...,2,1,0,0,19.30,16.83,19.97,18.01,21.83,17.60
6,Tina Zhu,10,"3,4,5,6,7,8,11,12,15,16",6,5,6,6,0,0,0,...,0,0,0,0,9.16,-2.70,16.21,10.27,14.44,6.48
7,#19 Ben Wang,4,"2,11,15,16",2,2,1,2,0,0,0,...,1,0,0,0,7.58,-2.72,0.00,0.00,23.72,23.64
8,Theresa Zhang,4,"9,10,13,14",0,0,0,0,0,0,0,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00
9,#66 Jerry Chen,2,"5,12",0,0,0,0,0,0,0,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00


In [27]:
player_stats_df['Player']

KeyError: 'Player'

In [4]:
if None:
    print('hello')